In [ ]:
import rospy
import cv2
import numpy as np
import os
from sensor_msgs.msg import Image
from std_msgs.msg import String
from darknet_ros_msgs.msg import BoundingBoxes
from pathlib import Path

class FCW:
    def __init__(self, video_file):
        rospy.init_node('fcw_node')

        # Initialize video capture
        self.video_capture = cv2.VideoCapture(video_file)
        if not self.video_capture.isOpened():
            rospy.logerr("Failed to open video file.")
            return

        self.output_video_path = "output-video.mp4"
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.video_writer = cv2.VideoWriter(self.output_video_path, fourcc, 30.0, (640, 480))

        # Load dataset (assuming the dataset has a folder of images or annotations)
        self.dataset_path = "source/directory/bbd100k"
        if not os.path.exists(self.dataset_path):
            rospy.logerr(f"Dataset path {self.dataset_path} does not exist.")
            return

        rospy.Subscriber('/darknet_ros/bounding_boxes', BoundingBoxes, self.yolo_callback)
        self.warning_pub = rospy.Publisher('/fcw_warning', String, queue_size=10)
        self.min_distance_threshold = 5.0

    def yolo_callback(self, yolo_data):
        for box in yolo_data.bounding_boxes:
            if box.Class == 'car':
                img_width = 640  # Replace with actual image width
                center_x = (box.xmin + box.xmax) / 2.0
                distance = self.estimate_distance(center_x, img_width)
                if distance <= self.min_distance_threshold:
                    self.warning_pub.publish("COLLISION WARNING!")
                    break

    def estimate_distance(self, center_x, img_width):
        # This is a placeholder for a more sophisticated distance estimation
        return (img_width / 2 - center_x) * 0.1 + 2.0

    def process_video(self):
        # Read each frame from the video file
        while self.video_capture.isOpened():
            ret, frame = self.video_capture.read()
            if not ret:
                break

            # Perform bounding box detection (for now, using dummy data)
            dummy_bounding_boxes = BoundingBoxes()
            dummy_box = BoundingBoxes.BoundingBox()
            dummy_box.Class = 'car'
            dummy_box.xmin = 100
            dummy_box.ymin = 100
            dummy_box.xmax = 300
            dummy_box.ymax = 300
            dummy_bounding_boxes.bounding_boxes.append(dummy_box)

            self.yolo_callback(dummy_bounding_boxes)

            # Add the processed frame to the output video
            self.video_writer.write(frame)

        # Release video resources
        self.video_capture.release()
        self.video_writer.release()
        rospy.loginfo(f"Processed video saved as {self.output_video_path}")


import unittest
from unittest.mock import patch

class TestFCW(unittest.TestCase):
    @patch('forward_collision_warning.FCW.estimate_distance')
    def test_warning_triggered(self, mock_estimate_distance):
        fcw = FCW("sample_video.mp4")
        mock_estimate_distance.return_value = 4.0
        yolo_data = BoundingBoxes()
        box = BoundingBoxes.BoundingBox()
        box.Class = 'car'
        yolo_data.bounding_boxes.append(box)
        with patch('rospy.Publisher.publish') as mock_publish:
            fcw.yolo_callback(yolo_data)
            mock_publish.assert_called_with("COLLISION WARNING!")

    @patch('forward_collision_warning.FCW.estimate_distance')
    def test_warning_not_triggered(self, mock_estimate_distance):
        fcw = FCW("sample_video.mp4")
        mock_estimate_distance.return_value = 6.0
        yolo_data = BoundingBoxes()
        box = BoundingBoxes.BoundingBox()
        box.Class = 'car'
        yolo_data.bounding_boxes.append(box)
        with patch('rospy.Publisher.publish') as mock_publish:
            fcw.yolo_callback(yolo_data)
            mock_publish.assert_not_called()

    def test_estimate_distance(self):
        fcw = FCW("sample_video.mp4")
        self.assertAlmostEqual(fcw.estimate_distance(320, 640), 2.0)
        self.assertAlmostEqual(fcw.estimate_distance(200, 640), 4.0)

if __name__ == '__main__':
    # Initialize the FCW node with the path to the input video file
    fcw_node = FCW("sample_video.mp4")

    # Process the video and save the output
    fcw_node.process_video()

    # Run unit tests
    unittest.main()
